In [9]:
import json
import numpy as np

In [10]:
f = open('giga_annotations/person_bbox_train.json')
person_data = json.load(f)

In [11]:
coco_images = []
person_ann = []
ann_id = 1
for key in person_data.keys():
    image_height = int(person_data[key]['image size']['height'])
    image_width = int(person_data[key]['image size']['width'])
    image_id = person_data[key]['image id']
    coco_images.append({"file_name" : key.split('/')[1], 
                        "id" : image_id,
                        "license" : 1,
                        "url" : "",
                        "height" : image_height,
                        "width" : image_width})
    for item in person_data[key]['objects list']:
        if item['category']!='person':
            xmin=int(item['rect']['tl']['x']*image_width)
            ymin=int(item['rect']['tl']['y']*image_height)
            xmax=int(item['rect']['br']['x']*image_width)
            ymax=int(item['rect']['br']['y']*image_height)
        else:
            xmin=int(item['rects']['full body']['tl']['x']*image_width)
            ymin=int(item['rects']['full body']['tl']['y']*image_height)
            xmax=int(item['rects']['full body']['br']['x']*image_width)
            ymax=int(item['rects']['full body']['br']['y']*image_height)
        w_bbox=xmax-xmin
        h_bbox=ymax-ymin
        person_ann.append({"segmentation" : [],
                            "area" : np.float64(w_bbox*h_bbox),
                            "iscrowd" : 0,
                            "image_id" : image_id,
                            "bbox" : [xmin, ymin, w_bbox, h_bbox],
                            "category_id" : 2,
                            "id": ann_id})
        ann_id += 1

In [12]:
f = open('giga_annotations/vehicle_bbox_train.json')
vehicle_data = json.load(f)

In [13]:
vehicle_ann = []
for key in vehicle_data.keys():
    image_height = int(vehicle_data[key]['image size']['height'])
    image_width = int(vehicle_data[key]['image size']['width'])
    image_id = vehicle_data[key]['image id']
    for item in vehicle_data[key]['objects list']:
        xmin=int(item['rect']['tl']['x']*image_width)
        ymin=int(item['rect']['tl']['y']*image_height)
        xmax=int(item['rect']['br']['x']*image_width)
        ymax=int(item['rect']['br']['y']*image_height)
        w_bbox=xmax-xmin
        h_bbox=ymax-ymin
        vehicle_ann.append({"segmentation" : [],
                            "area" : np.float64(w_bbox*h_bbox),
                            "iscrowd" : 0,
                            "image_id" : image_id,
                            "bbox" : [xmin, ymin, w_bbox, h_bbox],
                            "category_id" : 1,
                            "id": ann_id})
        ann_id += 1

In [14]:
annotations = []
annotations.extend(person_ann)
annotations.extend(vehicle_ann)
print(len(annotations))

122865


In [15]:
coco={}
coco["info"] = {"year" : "2023",
                "version" : "1.0",
                "description" : "Giga Vision COCO",
                "contributor" : "DilisionAI",
                "url" : "",
                "date_created" : "19 Jan"
                }
coco["licenses"] = [{"id": 1,
                  "name": "Attribution-NonCommercial",
                  "url": "http://creativecommons.org/licenses/by-nc-sa/2.0/"
                 }]

coco["categories"] = [{'supercategory': 'person', 'id': 2, 'name': 'person'},
                     {'supercategory': 'vehicle', 'id': 1, 'name': 'vehicle'}]
coco['images'] = coco_images
coco['annotations'] = annotations
with open('giga_train_coco.json', 'w') as json_file:
    json.dump(coco, json_file, sort_keys=True, indent=4)